# Cosmosdb connection testing

## Azure python documentation
https://learn.microsoft.com/en-us/python/api/azure-cosmos/azure.cosmos?preserve-view=true&view=azure-python


In [15]:
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
import os
import json
from dotenv import load_dotenv
from azure.cosmos import CosmosClient, PartitionKey

load_dotenv('../../shared/.env')

True

In [16]:
# database env
URL = os.getenv('COSMOSDB_ENDPOINT')  
MASTER_KEY = os.getenv('COSMOSDB_KEY')

In [17]:
# init client
client = cosmos_client.CosmosClient(URL, {'masterKey': MASTER_KEY}, user_agent="", user_agent_overwrite=True)


In [18]:
# Create a database (if it doesn't exist)
database_name = 'MojovaDB'
database = client.get_database_client(database_name)

# Create a container (if it doesn't exist)
container_name = 'Analytics'
try:
    container = database.create_container_if_not_exists(
        id=container_name,
        partition_key=PartitionKey(path='/medallion'),
        offer_throughput=400
    )
except exceptions.CosmosResourceExistsError:
    container = database.get_container_client(container_name)

In [5]:
# read local datafile
# print first json element
datafile = '../../data/ingestion/117_HOpp-Q3_2023.xlsx.json'

with open(datafile, "r") as f:
    content = f.read()
    content = json.loads(content)
    print(content[0])


{'organisaatiokoodi_kolme_numeroa': 117, 'yksikkokoodiks_luokitteluohje_yksikkokoodit-valilehdelta': 'AIKKIRMUU', 'kvartaali_ja_vuosiesim_1_2020': '3_2023', '1_hoitajat_ottivat_mielipiteeni_huomioon_kun_hoitoani_suunniteltiin_tai_toteutettiin': 5, '2_hoitajat_ja_laakarit_toimivat_hyvin_yhdessa_hoitooni_liittyvissa_asioissa': 5, '3_hoitoni_oli_hyvin_suunniteltu_ja_toteutettu_hoitajien_seka_laakareiden_toimesta': 5, '4_hoitajat_pyysivat_minulta_anteeksi_jos_hoidossani_tapahtui_virhe': 5, '5_hoitajat_puuttuivat_epakohtaan_josta_mainitsin_heille': 5, '6_hoitajat_kertoivat_minulle_uuden_laakkeen_antamisen_yhteydessa_miksi_laaketta_annetaan': 5, '7_hoitajat_kertoivat_minulle_saamieni_laakkeiden_mahdollisista_sivuvaikutuksista': 5, '8_hoitajat_puhuivat_arkaluontoisista_asioista_siten_etteivat_ulkopuoliset_kuulleet_niita': 5, '9_hoitajat_huolehtivat_etta_liikkuminen_oli_turvallista_hoidon_aikana': 5, '10_hoitajat_kohtelivat_minua_hyvin': 5, '11_hoitajat_huolehtivat_etteivat_hoito_ja_tai_tutkim

In [6]:
# add data to container
import uuid
for item in content:
    item.setdefault("id", str(uuid.uuid4()))
    container.upsert_item(item)

In [38]:
import pandas as pd
query = "SELECT * FROM c WHERE c['/medallion'] = 'bronze_nes' AND c.tyoyksikko = 'ENSIHOITO'"
data = container.query_items(query=query, enable_cross_partition_query=True)
# data contasins iterator object
data = [item for item in data]


In [39]:
df = pd.DataFrame(data)
df

,org_id,unnamed:_1,tyoyksikko,tyonkuva,unnamed:_4,tyosuhde,koulutus,tyovuoro,tyosuhteen_pituus,uskon_org_paamaariin,...,org_innostaa,tyoskentelen_3v_todnak,valmis_panostamaan,id,/medallion,_rid,_self,_etag,_attachments,_ts
0,117,Ensihoito,ENSIHOITO,16,"Sairaanhoitaja, audionomi, terveydenhoitaja",1,3,3.0,5,5,...,3,5,5,f05d352f-857b-4367-aa74-afba58d78e79,bronze_nes,1kAqANsuEko1AAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f005621-0000-4700-0000-6730a7c30000""",attachments/,1731241923
1,117,Ensihoito,ENSIHOITO,20,Ensihoitaja / vastaava ensihoitaja,1,3,3.0,4,4,...,4,4,5,e057e003-f7d3-414a-af6d-d10e82bf5849,bronze_nes,1kAqANsuEkpNAAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f006e21-0000-4700-0000-6730a7c40000""",attachments/,1731241924
2,117,Ensihoito,ENSIHOITO,20,Ensihoitaja / vastaava ensihoitaja,1,3,3.0,4,2,...,2,4,4,d4c5e1a8-43dd-43f4-a32a-01360c7c3c14,bronze_nes,1kAqANsuEkpWAAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f007c21-0000-4700-0000-6730a7c40000""",attachments/,1731241924
3,117,Ensihoito,ENSIHOITO,20,Ensihoitaja / vastaava ensihoitaja,2,3,3.0,2,5,...,3,3,5,49776cde-5ebb-4478-8879-085e29ddd11e,bronze_nes,1kAqANsuEkp1AAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f00a121-0000-4700-0000-6730a7c60000""",attachments/,1731241926
4,117,Ensihoito,ENSIHOITO,16,"Sairaanhoitaja, audionomi, terveydenhoitaja",1,3,NaN,5,4,...,5,6,6,8fd043dd-a16e-4ebe-b7ad-60580e4e93a1,bronze_nes,1kAqANsuEkqIAAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f00b621-0000-4700-0000-6730a7c80000""",attachments/,1731241928
5,117,Ensihoito,ENSIHOITO,20,Ensihoitaja / vastaava ensihoitaja,1,3,3.0,4,5,...,2,3,4,4e4ff758-27c8-4d91-9bc9-adeb4413d9a4,bronze_nes,1kAqANsuEkrAAAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f00f521-0000-4700-0000-6730a7cc0000""",attachments/,1731241932
6,117,Ensihoito,ENSIHOITO,10,"Lähihoitaja, perushoitaja, lvm-sairaankuljetta...",1,5,3.0,5,5,...,6,6,6,0339344f-d543-4601-a09a-15c61c3c661c,bronze_nes,1kAqANsuEkrBAAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f00f621-0000-4700-0000-6730a7cc0000""",attachments/,1731241932
7,117,Ensihoito,ENSIHOITO,20,Ensihoitaja / vastaava ensihoitaja,1,3,3.0,5,2,...,1,4,5,b29e0927-a8d0-4405-be9d-af913009e2ee,bronze_nes,1kAqANsuEkrcAAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f001322-0000-4700-0000-6730a7ce0000""",attachments/,1731241934
8,117,Ensihoito,ENSIHOITO,20,Ensihoitaja / vastaava ensihoitaja,1,3,3.0,4,2,...,1,2,4,735afb1d-4328-4f87-b316-9a546859cb91,bronze_nes,1kAqANsuEkrqAAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f002122-0000-4700-0000-6730a7ce0000""",attachments/,1731241934
9,117,Ensihoito,ENSIHOITO,20,Ensihoitaja / vastaava ensihoitaja,1,3,NaN,4,4,...,3,2,4,c3d33147-0c8f-4cc1-b111-47aa042017c8,bronze_nes,1kAqANsuEkrsAAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f002322-0000-4700-0000-6730a7cf0000""",attachments/,1731241935
